In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import json

In [2]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [3]:
users_reviews = []
for l in parse("review-Hawaii_10.json.gz"):
  users_reviews.append(l)



In [5]:
print(f"users_reviews dataset length: {len(users_reviews)}")
print("users_reviews dataset example:")
users_reviews[0]

users_reviews dataset length: 1504347
users_reviews dataset example:


{'user_id': '113965417079576625433',
 'name': 'manuel grimaldo',
 'time': 1591839903487,
 'rating': 5,
 'text': 'Great new upgrade',
 'pics': None,
 'resp': None,
 'gmap_id': '0x7c00159b5b1b1d25:0x8d2d85d4a758290e'}

In [6]:
businesses = []
for l in parse("meta-Hawaii.json.gz"):
  businesses.append(l)

In [7]:
print(f"businesses dataset length: {len(businesses)}")
print("businesses dataset example:")
businesses[0]

businesses dataset length: 21507
businesses dataset example:


{'name': 'Hale Pops',
 'address': 'Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762',
 'gmap_id': '0x7c00456eecad3111:0x8217f9600c51f33',
 'description': None,
 'latitude': 21.637795699999998,
 'longitude': -157.9207142,
 'category': ['Restaurant'],
 'avg_rating': 4.4,
 'num_of_reviews': 18,
 'price': None,
 'hours': [['Thursday', '11AM–8PM'],
  ['Friday', '11AM–8PM'],
  ['Saturday', '11AM–8PM'],
  ['Sunday', 'Closed'],
  ['Monday', '11AM–8PM'],
  ['Tuesday', '11AM–8PM'],
  ['Wednesday', '11AM–8PM']],
 'MISC': {'Service options': ['Outdoor seating', 'Takeout', 'Delivery'],
  'Popular for': ['Lunch', 'Solo dining'],
  'Accessibility': ['Wheelchair accessible entrance'],
  'Offerings': ['Comfort food', 'Quick bite'],
  'Amenities': ['Good for kids'],
  'Atmosphere': ['Casual'],
  'Crowd': ['Groups', 'Tourists'],
  'Payments': ['NFC mobile payments']},
 'state': 'Closed ⋅ Opens 11AM',
 'relative_results': ['0x7c00451360f80cf1:0x930291a38bab3132',
  '0x7c00457322571d57:0xe3974e89bbfc41ce',

In [10]:
business_user = defaultdict(list)
for d in users_reviews:
    business_user[d['gmap_id']].append(d['user_id'])

# print number of business
print(f"number of business: {len(business_user)}")

number of business: 11686


In [8]:
# extract all business id from businesses
business_id = []
for d in businesses:
    business_id.append(d['gmap_id'])

In [11]:
# check if every business in users dataset is in business dataset
for b in business_user:
    if b not in business_id:
        print(f"business {b} not in business dataset")